# Exploratory Data Analysis

This notebook contains the exploratory data analysis for the bank fraud detection dataset. It is split into multiple sections being 

- Initial understanding of the data
- Exploring the features individually (visualization/outliers)
    - Numeric data
    - Numerical categorical data
    - Categoricla data
- Exploring the features in relation to eachother (visualization/correlation)
- Identify possible transformations 

### 1. Initial understanding of the data

In [4]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# Data collection
total_df = pd.read_csv('./Data/Base.csv')

# Split the DataFrame into training and test sets using stratified sampling to maintain anomaly distribution
train_df, test_df = train_test_split(total_df, test_size=0.2, stratify=total_df['fraud_bool'], random_state=42)

# Validate the size of the data
train_shape = train_df.shape
test_shape = test_df.shape
print(f"The training data has {train_shape[0]} rows and {train_shape[1]} columns.")
print(f"The testing data has {test_shape[0]} rows and {test_shape[1]} columns.")

The training data has 800000 rows and 32 columns.
The testing data has 200000 rows and 32 columns.


We start the EDA by importing data from the Base.csv file and split this up into training and test data to avoid data leakage. The split is done using stratified sampling from the "fraud_bool" feature which preserves output distribution due to most likely unbalanced features. We can see that the data has 32 columns where the training data contains 80000 samples and the test data has 20000. Next we can have a quick look at what the dataframe looks like.

In [8]:
# Shows the first 5 observations of the training data
train_df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
39111,0,0.7,0.229712,-1,63,50,0.024720,50.674001,AA,1305,...,1,1500.0,0,INTERNET,3.580550,linux,0,1,0,0
822700,0,0.2,0.928428,199,24,70,0.014153,15.631407,AA,833,...,0,500.0,0,INTERNET,7.087779,other,1,1,0,6
914415,0,0.1,0.658630,95,2,40,0.045801,-1.410133,AB,237,...,0,200.0,0,INTERNET,0.547804,other,1,1,0,7
581307,0,0.8,0.774858,-1,122,30,0.005569,-0.539938,AB,895,...,1,500.0,0,INTERNET,4.671407,other,1,1,0,4
603136,0,0.9,0.993460,103,9,20,0.010832,-0.501067,AB,4105,...,1,200.0,0,INTERNET,9.293206,linux,0,1,0,4


Above, we can see that the data consists of a target variable called fraud_bool along with many features which give information on whether a certain bank account is fraudulent. The name fraud_bool hints at it being a boolean feature so we can check the different possible outputs in the fraud_bool feature and their distribution by doing the following

In [15]:
# Find the unique values of fraud_bool and prints their distribution
unique_outputs= train_df['fraud_bool'].unique()
print(f"The unique values in the column are: {', '.join(map(str, unique_outputs))}")

# Get the distribution of the fraud_bool
distribution = df['fraud_bool'].value_counts(normalize=True)
print("\nDistribution of the output variable:")
for value, proportion in distribution.items():
    print(f"Value {value}: {proportion:.2%}")



The unique values in the column are: 0, 1

Distribution of the output variable:
Value 0: 98.90%
Value 1: 1.10%


Now we see us that our problem is a binary classification problem because we only have two outputs 0 and 1. We also see that our data is highly imbalanced as 98.9% of the data is not fraudulent whereas 1.1% is a fraud. This will need to be taken into account for the preprocessing of the data. Moving on we can calculate the types of all of the features to determine what we are dealing with

In [9]:
# Print the column names, non-null count and datatype
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 39111 to 228494
Data columns (total 32 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   fraud_bool                        800000 non-null  int64  
 1   income                            800000 non-null  float64
 2   name_email_similarity             800000 non-null  float64
 3   prev_address_months_count         800000 non-null  int64  
 4   current_address_months_count      800000 non-null  int64  
 5   customer_age                      800000 non-null  int64  
 6   days_since_request                800000 non-null  float64
 7   intended_balcon_amount            800000 non-null  float64
 8   payment_type                      800000 non-null  object 
 9   zip_count_4w                      800000 non-null  int64  
 10  velocity_6h                       800000 non-null  float64
 11  velocity_24h                      800000 non-nul

Here we get the information that we have no null features or missing features meaning that data imputation won't be necessary for our project. We have numerical catergorical features with the type "int64", numerical features with the type "float64" and a few categorical features with the type "object". Let's create 3 seperate dataframes for these different feature categories and study them seperately

In [16]:
# Separate DataFrames based on column data types
num_df = train_df.select_dtypes(include=['float64'])  # Numerical data 
numcat_df = train_df.select_dtypes(include=['int64'])  # Numerical categorical
cat_df = train_df.select_dtypes(include=['object'])     # Categorical 

## 2. Exploring the features individually

### 2.1 Numerical features

We start by analysing the numerical features. The first thing we can check is the statistics of the data such as the mean and standard deviations. 

In [18]:
num_df.describe()

,income,name_email_similarity,days_since_request,intended_balcon_amount,velocity_6h,velocity_24h,velocity_4w,proposed_credit_limit,session_length_in_minutes
count,800000.000000,800000.000000,8.000000e+05,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000
mean,0.562860,0.493798,1.024099e+00,8.665099,5664.020229,4770.234656,4856.003621,516.090400,7.549768
std,0.290343,0.289099,5.377299e+00,20.232373,3009.677961,1479.503976,919.619696,487.736585,8.044859
min,0.100000,0.000001,4.036860e-09,-15.530555,-170.603072,1320.283991,2825.748405,190.000000,-1.000000
25%,0.300000,0.225325,7.181675e-03,-1.181143,3434.759967,3593.073983,4268.308917,200.000000,3.104958
50%,0.600000,0.492314,1.516505e-02,-0.829849,5316.302685,4750.803340,4913.542421,200.000000,5.113827
75%,0.800000,0.755595,2.631458e-02,5.074825,7680.990796,5753.115565,5487.683683,500.000000,8.866801
max,0.900000,0.999999,7.845690e+01,112.956928,16715.565404,9506.896596,6994.764201,2100.000000,85.899143


Looking the table above we can see that the features called velocity have a very high mean and that the feature proposed_credit_limit also is quite high. When reading the documentation about the dataset we learn that velocity is the average number of applications per hour during the stated time in the feature (velocity_6h is the last 6 hours). One can already speculate that perhaps looking 4 weeks back when averaging per hour won't be a necessary feature since it is over such a long time span that frauds will cancel out in the mix. Another noticiable thing is that the standard deviation of days_since_request is very large indicating that this feature could be close to a uniform distribution and probably also isn't relevant for our model. Another interesting aspect is that the minimum value of session_length_in_minutes is -1 which could mean that this value is a placeholder for something else since negative time isn't possible. Further analysis will need to be done on the features individually to determine proper transformations that should be done to the dataset

TODO: Analyse the features one by one and visualise them with histograms and boxplots